In [1]:
import sys, os
sys.path.append("..")
os.chdir("..")
import torch, torch_geometric as pyg

print("PyTorch:", torch.__version__, *torch.__path__)
print("PyG:", pyg.__version__, *pyg.__path__)
print("CPU parallelism:", torch.get_num_threads())


PyTorch: 2.0.1+cu117 /home/tianhaoh/.local/lib/python3.9/site-packages/torch
PyG: 2.3.1 /home/tianhaoh/.local/lib/python3.9/site-packages/torch_geometric
CPU parallelism: 12


In [2]:
from torch_geometric.utils import index_to_mask, mask_to_index
from data.ops import scatter_append
from data.partitioner import (
    RandomNodePartitioner, MetisWeightedPartitioner,
    FennelPartitioner, FennelStrataPartitioner, ReFennelPartitioner,
)
from graphutils.rw import lazy_rw

from pathlib import Path
from ogb.nodeproppred import PygNodePropPredDataset
import torch_geometric.transforms as T

path = Path('/opt/datasets/')
dataset = PygNodePropPredDataset(
    root=path, name='ogbn-arxiv',
    pre_transform=T.ToUndirected(),
    transform=T.ToSparseTensor()
)
data = dataset[0]
data.NID = torch.arange(0, data.num_nodes, dtype=torch.int32)
num_classes = dataset.num_classes
print(data)
split = dataset.get_idx_split()
train_nid, val_nid, test_nid = split['train'], split['valid'], split['test']
print(f"train: {len(train_nid)}, val: {len(val_nid)}, test: {len(test_nid)}")
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class FennelStrataOrderPartitioner(FennelStrataPartitioner):
    def __init__(self, g, psize, name='Fennel-strata-deg', **kwargs):
        super().__init__(g, psize, name=name, **kwargs)
        # overwrite node_order
        self.node_order = torch.arange(g.size(0))


Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=2315598], NID=[169343])
train: 90941, val: 29799, test: 48603


In [12]:
from torch_geometric.utils import degree
nids = torch.arange(1)
n = data.num_nodes
init_score = torch.zeros((data.num_nodes,), device=nids.device)
init_score[data.NID] = 1 / data.NID.size(0)
final_score = lazy_rw(data.adj_t, init_score, k=1)
print(final_score)
print(final_score.sum())
# print("score sum:", final_score.sum(), "topk sum:", topk.values.sum())

tensor([4.8885e-05, 4.6398e-06, 5.0517e-06,  ..., 4.5261e-06, 4.2238e-06,
        3.0427e-06])
tensor(1.0000)


In [3]:
P = 64
nodes = data.NID
labels = data.y.flatten().clone()
num_labels = labels.int().max() + 0
train_mask = index_to_mask(train_nid, data.size(-1))
labels[~train_mask] = num_labels
FennelP = ReFennelPartitioner(
            data, psize=P, slack=1.5, alpha_ratio=0.1, beta=1, runs=5,
            base=FennelPartitioner,
        )
FennelLB = ReFennelPartitioner(
            data, psize=P, slack=1.5, alpha_ratio=0.1, beta=1, runs=5,
            base=FennelStrataOrderPartitioner,
            labels=labels,
        )
MetisP = MetisWeightedPartitioner(data, psize=P, node_weights=(labels.float()/4).int())
RandP = RandomNodePartitioner(data, psize=P)

assigns_mts = MetisP.partition()
assigns_fnl = FennelP.partition()
assigns_flb = FennelLB.partition()
assigns_rnd = RandP.partition()
parts_mts, ints_mts, _ = scatter_append(-1, assigns_mts, nodes, P)
parts_fnl, ints_fnl, _ = scatter_append(-1, assigns_fnl, nodes, P)
parts_flb, ints_flb, _ = scatter_append(-1, assigns_flb, nodes, P)
parts_rnb, ints_rnd, _ = scatter_append(-1, assigns_rnd, nodes, P)

def edge_cuts(data, assigns):
    src, dst, _ = data.adj_t.coo()
    return (assigns[src]-assigns[dst] != 0).int().sum().item()
print(edge_cuts(data, assigns_mts))
print(edge_cuts(data, assigns_fnl))
print(edge_cuts(data, assigns_flb))
print(edge_cuts(data, assigns_rnd))
print(ints_flb[0], ints_flb[-1], data.size(0))

Convert a graph into a bidirected graph: 0.067 seconds, peak memory: 11.150 GB
Construct multi-constraint weights: 0.006 seconds, peak memory: 11.150 GB


[13:36:22] /opt/dgl/src/graph/transform/metis_partition_hetero.cc:78: Partition a graph with 169343 nodes and 2315598 edges into 64 parts and get 729069 edge cuts


Metis partitioning: 1.825 seconds, peak memory: 11.288 GB
ReFennel Run#0
ReFennel Run#1
ReFennel Run#2
ReFennel Run#3
ReFennel Run#4
ReFennel Run#0
ReFennel Run#1
ReFennel Run#2
ReFennel Run#3
ReFennel Run#4
1458138
979378
1132892
2279308
tensor(0) tensor(169343) 169343


In [4]:
data.cuda()
train_cuda = train_nid.cuda()

def influential(data, nids, k=3, topk=100):
    init_score = torch.zeros((data.num_nodes,), device=nids.device)
    init_score[nids] = 1 / nids.size(0)
    final_score = lazy_rw(data.adj_t, init_score, k=k)
    topk = final_score.cpu().topk(topk)
    # print("score sum:", final_score.sum(), "topk sum:", topk.values.sum())
    return topk.indices, topk.values

def importance(data, nids, k=3, topk=100):
    init_score = torch.zeros((data.num_nodes,), device=nids.device)
    init_score[nids] = 1 / nids.size(0)
    final_score = torch.zeros_like(init_score)
    score = init_score
    for _ in range(k):
        score = lazy_rw(data.adj_t, score, k=1)
        final_score += score
    topk = final_score.cpu().topk(topk)
    # print("score sum:", final_score.sum(), "topk sum:", topk.values.sum())
    return topk.indices, topk.values

int_starts, int_ends = ints_flb[:-1], ints_flb[1:]
node_parts = [parts_flb[int_starts[i] : int_ends[i]] for i in range(P)]
train_parts = [node_parts[i][train_mask[node_parts[i]]] for i in range(P)]

fnl_starts, fnl_ends = ints_fnl[:-1], ints_fnl[1:]
fnl_parts = [parts_fnl[fnl_starts[i] : fnl_ends[i]] for i in range(P)]
train_fnl_parts = [fnl_parts[i][train_mask[fnl_parts[i]]] for i in range(P)]

mts_starts, mts_ends = ints_mts[:-1], ints_mts[1:]
mts_parts = [parts_mts[mts_starts[i] : mts_ends[i]] for i in range(P)]
train_mts_parts = [mts_parts[i][train_mask[mts_parts[i]]] for i in range(P)]

# compare with random partitioning
rnd_starts, rnd_ends = ints_rnd[:-1], ints_rnd[1:]
rnd_parts = [parts_rnb[rnd_starts[i] : rnd_ends[i]] for i in range(P)]
train_rnd_parts = [rnd_parts[i][train_mask[rnd_parts[i]]] for i in range(P)]


In [5]:
# train_topk is a list of topk influential nodes for each training node
import os
if os.path.exists('notebooks/train_topk.pt'):
    train_topk, train_topk_scores = torch.load('notebooks/train_topk.pt')
else:
    train_topk = torch.empty((train_nid.size(0), 100), dtype=torch.long)
    train_topk_scores = torch.empty((train_nid.size(0), 100), dtype=torch.float)
    from tqdm import tqdm
    for i, t in enumerate(tqdm(train_cuda)):
        t_topk = influential(data, torch.tensor([t], device='cuda'), topk=100)
        train_topk[i][:] = t_topk[0]
        train_topk_scores[i][:] = t_topk[1]
    torch.save([train_topk, train_topk_scores], "notebooks/train_topk.pt")

In [8]:
import numpy as np
def intersect(s1, s2):
    return torch.from_numpy(np.intersect1d(s1.numpy(), s2.numpy()))

def get_coverage(train_topk, train_parts, node_parts, pivots):
    train_idx = torch.arange(train_nid.size(0))
    train_map = torch.zeros(data.size(0), dtype=torch.int64)
    train_map[train_nid] = train_idx

    best = []
    coverage = []
    for i in range(len(train_parts)):
        trains = train_parts[i]
        mask = torch.zeros(data.size(0), dtype=torch.bool)
        mask[node_parts[i]] = True
        mask[pivots] = True
        per_part: torch.Tensor = torch.zeros(data.size(0))
        scores = 0
        mapped = train_map[trains]
        for t in mapped:
            current_topk = train_topk[t]
            current_scores = train_topk_scores[t]
            contains_topk = mask[current_topk]
            scores += current_scores[contains_topk].sum()
            per_part[current_topk] += current_scores
        coverage.append(scores / trains.size(0))
        best.append(per_part.topk(node_parts[i].size(0)).values.sum().item() / trains.size(0))

    print("done")
    return coverage, best


In [14]:
pivots, _ = importance(data, train_cuda, k=3, topk=int(data.size(0)/32))
hubs, _ = influential(data, data.NID, k=1, topk=int(data.size(0)/32))
empty = torch.tensor([], dtype=torch.long)

# Fennel-LB
pivots_cov, pivots_best = get_coverage(train_topk, train_parts, node_parts, pivots)
hubs_cov, hubs_best = get_coverage(train_topk, train_parts, node_parts, hubs)
nop_cov, nop_best = get_coverage(train_topk, train_parts, node_parts, empty)

# pivots_fnl_cov, _ = get_coverage(train_topk, train_fnl_parts, fnl_parts, pivots)
# hubs_fnl_cov, _ = get_coverage(train_topk, train_fnl_parts, fnl_parts, hubs)
# nop_fnl_cov, _ = get_coverage(train_topk, train_fnl_parts, fnl_parts, empty)

pivots_mts_cov, pivots_mts_best = get_coverage(train_topk, train_mts_parts, mts_parts, pivots)
mts_cov, mts_best = get_coverage(train_topk, train_mts_parts, mts_parts, empty)
pivots_rnd_cov, pivots_rnd_best = get_coverage(train_topk, train_rnd_parts, rnd_parts, pivots)
nop_rnd_cov, nop_rnd_best = get_coverage(train_topk, train_rnd_parts, rnd_parts, empty)

done
done
done
done
done
done
done


In [15]:
def mean_std(hits):
    tensor = torch.tensor(hits)
    return torch.mean(tensor).item() # , torch.std(tensor).item()
print(
    f"random:\t\t\t{mean_std(nop_rnd_cov), mean_std(nop_rnd_best)}\n",
    f"random+pivot:\t\t{mean_std(pivots_rnd_cov), mean_std(pivots_rnd_best)}\n",
    f"fennel-LB:\t\t{mean_std(nop_cov), mean_std(nop_best)}\n",
    f"fennel-LB+pivot:\t{mean_std(pivots_cov), mean_std(pivots_best)}\n",
    f"fennel-LB+hubs:\t{mean_std(hubs_cov), mean_std(pivots_best)}\n",
    f"metis:\t\t\t{mean_std(mts_cov), mean_std(mts_best)}\n",
    f"metis+pivots:\t\t{mean_std(pivots_mts_cov), mean_std(pivots_mts_best)}\n",
)

random:			(0.19603095948696136, 0.41245684027671814)
 random+pivot:		(0.37055063247680664, 0.41245684027671814)
 fennel-LB:		(0.5217406153678894, 0.6659396290779114)
 fennel-LB+pivot:	(0.6288290619850159, 0.6659396290779114)
 fennel-LB+hubs:	(0.6199474930763245, 0.6659396290779114)
 metis:			(0.5713961720466614, 0.708052396774292)
 metis+pivots:		(0.6705724000930786, 0.708052396774292)



In [ ]:
train_idx = torch.arange(train_nid.size(0))
train_map = torch.zeros(data.size(0), dtype=torch.int64)
train_map[train_nid] = train_idx

def union(tensor_list, exclude=None):
    mask = torch.zeros(data.size(0))
    for t in tensor_list:
        mask[t] = True
    if exclude is not None:
        mask[exclude] = False
    return mask_to_index(mask)

topk = [union(train_topk[train_map[train_parts[i]]]) for i in range(P)]